<font size="6"><h3 align="center"><span style="color:red">Smallest Firewall table</span></h3></font>



This script uses an GUI to take user inputs. The inputs are:

1. Function - The options are:

 i.Smallest functionally equivalent firewall table
 
 ii.Frequently used Servers and Ports
 

2. Input File - Enter the name of input file

3. Output File - Enter a name that the user would like to assign to the output file

<span style="color:red">COMPLEXITY:</span> The complexity of the steps is noted in the markdown cells

## Importing the libraries

In [129]:
from collections import defaultdict
import PySimpleGUI as sg
import pandas as pd
import regex as re
import sys

## Creating the GUI

In [ ]:
#Return values as a list
window = sg.FlexForm('Simple data entry form')  # begin with a blank form

#Preparing the layout for the window screen
layout = [
          [sg.Text('Please enter the Function, Input File Name, Output File Name')],
          [sg.Text('The Functions available are:')],
          [sg.Text('1: Smallest functionally equivalent firewall table')],
          [sg.Text('2: Frequently used Servers and Ports')],
          [sg.Text('Function', size=(15, 1)), sg.InputText('function')],
          [sg.Text('Input File', size=(15, 1)), sg.InputText('input')],
          [sg.Text('Output File', size=(15, 1)), sg.InputText('output')],
          [sg.Submit()]
         ]

#Reading the events which would be the inputs in this case
button, values = window.Layout(layout).Read()
#Closing once the submit button is pressed
window.close()

## Validating inputs from the GUI

In [131]:
#If any of the inputs is empty, exit the program
if ((not values[0]) or ((not values[1])) or ((not values[2]))):
    print('Please fill all the inputs')
    sys.exit('Please fill all the inputs')

#Function should be either '1' or '2'. If not exit the program
if(values[0] not in ['1','2']):
    print('Options shouldbe either 1 or 2 only')
    sys.exit('Options shouldbe either 1 or 2 only')
    
#If the Input file isn't present, exit the program
try:
    df = pd.read_csv(values[1])
except FileNotFoundError:
    print('Input file does not exist')
    sys.exit('Input file does not exist')

## Step 1 - Merging redundant rules

This would remove rows which have similar server AND port number while retaining only their first occurance.

Ex:
Input-

R1	S1	22

R3	S1	22

R1	S2	80

R2	S2	80
 
Row 1 and 2 use the same servers and ports and therefore should be merged
Row 3 and 4 use the same servers and ports and therefore should be merged

Output-

R1	S1	22

R1	S2	80

<span style="color:red">COMPLEXITY:</span> Not sure about the O(n) notation but based on this thread, it's a faster approach than the other methods

https://stackoverflow.com/questions/13035764/remove-rows-with-duplicate-indices-pandas-dataframe-and-timeseries/34297689#34297689

In [114]:
def non_redundant_removal(dataframe):
    
    #Finding duplicates based on the server and port column
    non_redundant_df = dataframe[~dataframe.duplicated(['server', 'port'])].reset_index()
    non_redundant_df.drop('index', axis=1, inplace=True)
    return(non_redundant_df)

## Step 2 Consolidating rules for frequently occuring ports

In this step, the rule_name of the frequently occuring ports( port numbers repeating more than 1 time) will be consolidated

Ex Input -

R1	S1	22

R1	S2	22

R2	S2	22

R2	S3	22

R3	S1	22

R3	S3	22

Output - 

New R1 (row 1+5)	S1	22

New R1 (row 2+3)	S2	22

New R1 (row 4+6)	S3	22

<span style="color:red">COMPLEXITY:</span>

- Sorting dataframe by column uses quicksort :
Hence O(n log(n)) (best case) and O(n^2) (worst case)

- While loop :
O(n)

In [117]:
def port_consolidation(dataframe):
    
    #Sorting the dataframe by port in order to decrease complexity
    sorted_df = dataframe.sort_values(by=['port'], ascending=True).reset_index()
    sorted_df.drop('index', axis=1, inplace=True)
    
    #Running a single loop to check if the current port-no is equal to next port-no
    row = 0
    while (row < len(sorted_df)-1):
        #if the current and next port numbers are same, change the rule_name value based on the naming convention
        if (sorted_df.iloc[row,2] == sorted_df.iloc[row+1,2]):
            sorted_df.iloc[row,0]   = 'NR'+ str(sorted_df.iloc[row,2])
            sorted_df.iloc[row+1,0] = 'NR'+ str(sorted_df.iloc[row,2])
            #sorted_df['rule_name'][row + 1] = 'NR'+str(sorted_df['port'][row])
            row = row + 1
        else:
            row = row + 1
    
    return(sorted_df)

## Step 3  Joining the server with the smallest number of rules

In this step, similar servers will have similar rule_name

Ex Input: 

R1	S1	25

R2	S1	30

R3	S1	45

R4	S1	50

Output

New R1	S1	25

New R1	S1	30

New R1	S1	45

New R1	S1	50

<span style="color:red">COMPLEXITY:</span>

- Sorting dataframe by column uses quicksort :
Hence O(n log(n)) (best case) and O(n^2) (worst case)

- While loop :
O(n)

In [120]:
def server_smallest_rules(dataframe):
    
    ##Sorting the dataframe by server column in order to decrease complexity
    df_step3 = dataframe.sort_values(by=['server'], ascending=True).reset_index()
    df_step3.drop('index', axis=1, inplace=True)
    
    #Running a single loop to check if the current server-no is equal to next server-no
    row = 0
    while (row < len(df_step3)-1):
        #if the current and next server-no numbers are same, change the rule_name value based on the naming convention
        if (df_step3.iloc[row,1] == df_step3.iloc[row+1,1]):
            df_step3.iloc[row,0]   = 'NRS'+ re.findall(r'\d+',df_step3.iloc[row,1])[0]#str(sorted_df.iloc[row,2])
            df_step3.iloc[row+1,0] = 'NRS'+ re.findall(r'\d+',df_step3.iloc[row,1])[0]
            #sorted_df['rule_name'][row + 1] = 'NR'+str(sorted_df['port'][row])
            row = row + 1
        else:
            row = row + 1
            
    return(df_step3)

## Step 4  Make the rule have smallest number of ports

In this step, we find the smallest number of ports that allows the biggest number of servers in that rule. 

Ex: Input -

R-498	S-104	219

R-498	S-104	309

Output -

R-498	S-104	219

or 

R-498	S-104	309

but not both (in this code, only the first occurance would be retained)

<span style="color:red">COMPLEXITY:</span> Not sure about the O(n) notation but based on this thread, it's a faster approach than the other methods

https://stackoverflow.com/questions/13035764/remove-rows-with-duplicate-indices-pandas-dataframe-and-timeseries/34297689#34297689

In [122]:
def rule_smallest_ports(dataframe):
    
    #Finding duplicates based on the rule_name and server column
    df_step4 = dataframe[~dataframe.duplicated(['rule_name','server'])].reset_index()
    df_step4.drop('index', axis=1, inplace=True)
    
    return(df_step4)

## Step 4 directly after Step 2

Evaluating the number of rows if we skip step 3 and move directly to step 4

Used in the final funcion call

In [125]:
#step2_4_df = rule_smallest_ports(step2_df)

## Function to find out the most frequently used servers and ports

There can be times when the load on some particular servers or ports could be substainally high.

In that case, we would have to quickly identify those servers and ports in order to change settings

<span style="color:red">COMPLEXITY:</span>

- Defaultdict : O(n)

- Sorting dictionary based on key : O(n log n)

In [132]:
def Max_Server_Port_Frequency(dataframe):
    
    #Using defaultdict to store the frequency of all servers and ports
    Max_used_servers = defaultdict(int)
    Max_used_ports = defaultdict(int)
    for row in range(0,len(dataframe)):
        Max_used_servers[dataframe.iloc[row,1]] += 1
        Max_used_ports[dataframe.iloc[row,2]] += 1
    
    #Sorting both the dictionaries based on the key/frequency value in descending order and then selecting the top 10
    Max_used_servers_sorted = sorted(Max_used_servers.items(), key=lambda x: x[1],reverse = True)[0:10]
    Max_used_ports_sorted = sorted(Max_used_ports.items(), key=lambda x: x[1],reverse = True)[0:10]

    #Initializing 4 list to store the values which would be then transformed into a dataframe
    
    Server = []                   #Stores server name
    Server_frequency = []         #Stores server frequency
    port = []                     #Stores port name
    port_frequency = []           #Stores port frequency
    
    #Filling the empty lists
    for server_no,port_no in zip(Max_used_servers_sorted,Max_used_ports_sorted):
        Server.append(server_no[0])
        Server_frequency.append(server_no[1])
        port.append(port_no[0])
        port_frequency.append(port_no[1])
    
    #Creating a dataframe
    Server_Port_frequency = pd.DataFrame(list(zip(Server,Server_frequency,port,port_frequency)),columns =['Server','Server_frequency','port','port_frequency'])
    
    return(Server_Port_frequency)

## Implementing the function option(from the GUI)

In [ ]:
# option 1 produces the smallest functionally equivalent firewall table

if (values[0] == '1'):
    step1_df = non_redundant_removal(df)               #Step1
    step2_df = port_consolidation(step1_df)            #Step2
    step3_df_copy = step2_df.copy()                    #Creating a copy after step 2, this would be used to check the contradiction condition
    step3_df = server_smallest_rules(step3_df_copy)    #Step3
    step4_df = rule_smallest_ports(step3_df)           #Step4
    step2_4_df = rule_smallest_ports(step2_df)         #the dataframe obtained if we go from step2 -> 4 without using step 3

    #We will select the dataframe with the minimum number of rows
    if( len(step4_df) <= len(step2_4_df) ):            
        step4_df.to_csv(values[2],header=True, index=False)  #values[2] is the name of the output file
    else:
        step2_4_df.to_csv(values[2],header=True, index=False)
    
    #Verying if all the unique(non-duplicate) servers are retained
    if (len(set(step4_df['server'])) == len(set(df['server']))):   
        print('All servers from ' + str(values[1]) + ' retained in the smallest equivalent firwall table')

#If option chosen is 2 i.e frequency for max used servers and ports
elif(values[0] == '2'):
    
    Frequency_Dataframe = Max_Server_Port_Frequency(df)
    Frequency_Dataframe.to_csv(values[2],header=True, index=False)
    print(str(values[2]) + ' file created')